In [1]:
import numpy as np

returns = np.array([0.2, 0.12, 0.16, 0.25])
sigma1 = 0.06
sigma2 = 0.04
sigma3 = 0.07
sigma4 = 0.15

rho12 = 0.15
rho13 = 0.70
rho14 = -0.1
rho23 = 0.23
rho24 = 0.61
rho34 = -0.1

cov_matrix = np.array(
    [[sigma1**2, rho12*sigma1*sigma2, rho13*sigma1*sigma3, rho14*sigma1*sigma4],
     [rho12*sigma1*sigma2, sigma2**2, rho23*sigma2*sigma3, rho24*sigma2*sigma4],
     [rho13*sigma1*sigma3, rho23*sigma2*sigma3, sigma3**2, rho34*sigma3*sigma4],
     [rho14*sigma1*sigma4, rho24*sigma2*sigma4, rho34*sigma3*sigma4, sigma4**2]])

weights = np.array([0.25, 0.25, 0.25, 0.25])
portfolio_return = np.dot(returns, weights)
portfolio_std = np.dot(weights.T, np.dot(cov_matrix, weights))**0.5
portfolio_sharpe = portfolio_return / portfolio_std  # daily so no need to multiply by sqrt(252)

print(f"{portfolio_return=}")
print(f"{portfolio_std=}")
print(f"{portfolio_sharpe=}")

portfolio_return=0.1825
portfolio_std=0.05238558962157437
portfolio_sharpe=3.4837824928258434


#### One way to improve the Sharpe ratio is to adjust the weights, but the problem for fund managers is how.

In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt

base_dir = os.path.join("Database", 'US')
tickers = ["BRK-B", "PEP"]

brk = pd.read_csv(os.path.join(base_dir, "BRK-B.csv"), index_col=0)
pep = pd.read_csv(os.path.join(base_dir, "PEP.csv"), index_col=0)

brk.index = pd.to_datetime(brk.index)
pep.index = pd.to_datetime(pep.index)

# filter between 2020 and 2021
brk = brk[(brk.index.year >= 2020) & (brk.index.year <= 2021)]
pep = pep[(pep.index.year >= 2020) & (pep.index.year <= 2021)]

brk["return"] = brk["Close"].pct_change()
pep["return"] = pep["Close"].pct_change()

cov_matrix = np.cov(brk["return"], pep["return"])

# # find out the rolling covariance of the returns
# cov_matrix_66 = brk["return"].rolling(window=66).cov(pep["return"])
# cov_matrix_22 = brk["return"].rolling(window=22).cov(pep["return"])
# cov_matrix_7 = brk["return"].rolling(window=7).cov(pep["return"])

# find covariance of returns
cov_matrix_66 = brk["return"].ewm(span=66).cov(pep["return"])
cov_matrix_22 = brk["return"].ewm(span=22).cov(pep["return"])
cov_matrix_7 = brk["return"].ewm(span=7).cov(pep["return"])

cov_matrix_66.plot()
cov_matrix_22.plot()
cov_matrix_7.plot()

plt.legend(["66", "22", "7"])
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\USER\\Documents\\HKUST\\ISOM\\4520\\isom4520-hw-group1\\Database\\US\\BRK-B.csv'

In [15]:
returns = [0.162, 0.253, 0.15]
sigma1 = 5.72 / 100
sigma2 = 7.14 / 100
sigma3 = 6.89 / 100

correlation_matrix = np.array(
    [[1 * sigma1 * sigma1, 0.61 * sigma2 * sigma1, 0.23 * sigma3 * sigma1],
     [0.61 * sigma1 * sigma2, 1 * sigma2 * sigma2, 0.4 * sigma3 *  sigma2],
     [-0.23 * sigma1 * sigma3, -0.4 * sigma2 * sigma3, 1 * sigma3 * sigma3]])  # try flipping the signs of the first two weights in the third strategy

weights = np.array([0.333, 0.333, 0.333])

portfolio_return = np.dot(returns, weights)
portfolio_std = np.dot(weights.T, np.dot(correlation_matrix, weights)) ** 0.5
portfolio_sharpe = portfolio_return / portfolio_std

print(f"{portfolio_sharpe=}")

portfolio_sharpe=4.199659770206841


#### Insight: find a strategy that is uncorrelated with your current strategies, even it has a relatively low Sharpe ratio

In [16]:
def load_data(ticker: str) -> pd.DataFrame:
    dir = os.path.join("Database", 'US', f"{ticker}-1d-5.csv")
    df = pd.read_csv(dir, index_col=0)
    df.index = pd.to_datetime(df.index, utc = True)
    df.index = df.index.tz_convert('US/Eastern')

    return df

#### Betting against Beta
- Rationale: Long safe assets and short risky assets -> hold the position for at least a month

In [17]:
import statsmodels.api as sm

tickers_data = {}
for ticker in tickers:
    tickers_data[ticker] = load_data(ticker)
   
market = "US" 
market_data = load_data(market)

def calculate_beta(stock_returns, market_returns):
    X = sm.add_constant(market_returns)
    y = stock_returns
    model = sm.OLS(y, X)
    results = model.fit()
    raise NotImplementedError

def add_beta(ticker_data, market_data: pd.DataFrame):
    period = 132
    all_beta = [None for _ in range(period - 1)]
    
    merged_df = pd.merge(
        ticker_data["Close"],
        market_data["Close"],
        how="left",
        left_index=True,
        right_index=True,
        suffixes=("_stock", "_market")
    )
    
    for i, date in enumerate(merged_df.index):
        if i < period:
            continue
        stock_returns = merged_df.iloc[i - period:i]["Close_stock"].pct_change()
        market_returns = merged_df.iloc[i - period: i]["Close_market"].pct_change()
        
        stock_returns.dropna(inplace=True)
        market_returns.dropna(inplace=True)
        
        beta, iv = calculate_beta(stock_returns, market_returns)
        all_beta[date] = beta
    
    all_beta = pd.Series(all_beta)
    all_beta.name = "Beta"
    all_beta.index = pd.to_datetime(all_beta.index, utc=True)
    all_beta.index = all_beta.index.tz_convert("US/Eastern")

for ticker, ticker_data in tickers_data.items():
    print(f"{ticker} is about to be processed")
    add_beta(ticker_data, market_data)
    print(f"\t\u2713 {ticker} has been processed successfully")
    


FileNotFoundError: [Errno 2] No such file or directory: 'Database\\US\\BRK-B-1d-5.csv'

In [ ]:
start_dates = []
end_dates = []
df = ticker_data["AAPL"]

# get start and end date for each month in each year
for year in ticker_data["AAPL"].index.year.unique():
    for month in range(1, 13):
        df_tmp = df[(df.index.year == year) & (df.index.month == month)]
        
        if len(df_tmp) == 0:
            continue
        
        start_dates.append(df_tmp.index[0])
        end_dates.append(df_tmp.index[-1])
        
